In [2]:
import numpy as np
import math
import sympy as sp



In [3]:
#CALCULO TUBERIAS EN SERIE
#constantes fisicas
g = 9.81 #gravedad
u = 1.14 * 10**(-6) # coeficiente de viscosidad cinematica 15 grados

In [4]:
n = 1
p = 1 #densidad del fluido

Ht = 28.5 #altura total = z2-z1
E = 1
u

1.1399999999999999e-06

In [5]:
# datos a colocar (d(mm), L(m), k(mm), K)
d_tuberias = np.array([
    (600, 423, 0.3, 4.2),
    (500, 174, 0.0015, 3.4),
    (300, 373, 0, 5.3),
    (250, 121, 0, 7.5)
])
sp.Matrix(d_tuberias)

Matrix([
[600.0, 423.0,    0.3, 4.2],
[500.0, 174.0, 0.0015, 3.4],
[300.0, 373.0,    0.0, 5.3],
[250.0, 121.0,    0.0, 7.5]])

In [6]:
# datos tuberias caudal extremo final (l/s)
q_tuberias = np.array([
    60,
    74,
    60,
])/1000
sp.Matrix(q_tuberias)

Matrix([
[ 0.06],
[0.074],
[ 0.06]])

In [7]:
"""#Calculo perdidas de friccion
def p_friccion():
    d = 0
    h_f = np.empty(len(d_tuberias))
    for i in range(len(d_tuberias)):
        d += d_tuberias[i,1]/((d_tuberias[i,0]/1000)**5)
    for i in range(len(d_tuberias)):
        h_f[i] = Ht * ((d_tuberias[i,1]/(d_tuberias[i,0]/1000)**5)/d)

    return h_f

a = p_friccion()
a"""

'#Calculo perdidas de friccion\ndef p_friccion():\n    d = 0\n    h_f = np.empty(len(d_tuberias))\n    for i in range(len(d_tuberias)):\n        d += d_tuberias[i,1]/((d_tuberias[i,0]/1000)**5)\n    for i in range(len(d_tuberias)):\n        h_f[i] = Ht * ((d_tuberias[i,1]/(d_tuberias[i,0]/1000)**5)/d)\n\n    return h_f\n\na = p_friccion()\na'

In [8]:
def h_f1():
    d = 0
    for i in range(len(d_tuberias)):
        d += d_tuberias[i,1]/((d_tuberias[i,0]/1000)**5)
    h_f = Ht * (d_tuberias[0,1]/(d_tuberias[0,0]/1000)**5)/d
    return h_f
h_f = h_f1()
h_f

0.5375501984623472

In [64]:
# formula velocidad tuberia 1
D1_metros = d_tuberias[0,0]/1000
k1_metros = d_tuberias[0,2]/1000

def v_1():
    m = math.sqrt(2*g*D1_metros*h_f)
    v_1 = ((-2 * m)/math.sqrt(d_tuberias[0,1]))*\
          math.log10((k1_metros/(3.7*D1_metros))+\
                     (2.51*u*math.sqrt(d_tuberias[0,1]))/(D1_metros*m))
    return v_1


v_1 = v_1()
v_1

0.9195652130414507

In [10]:
# calculo perdidas menores tuberia 1
h_m1 = d_tuberias[0,3] * v_1**2/(2*g)
h_m1

0.18101532927375474

In [11]:
# calculo caudal tuberia 1
Q_1 = math.pi/4 * D1_metros**2 * v_1
Q_1

0.2600009386008979

In [60]:
#calculo de caudales

Q_n = np.empty([len(q_tuberias)+1])
Q = 0
Q_n[0] = Q_1

for i in range(len(q_tuberias)):
    Q += q_tuberias[i]
    Q_n[i+1] = Q_1-Q


sp.Matrix(Q_n)

Matrix([
[ 0.260000938600898],
[ 0.200000938600898],
[ 0.126000938600898],
[0.0660009386008979]])

In [58]:

v_n = np.empty(len(Q_n)+1)
v_n[0] = v_1

for i in range(len(Q_n)):
    v = 4 * Q_n[i]/(math.pi*(d_tuberias[i,0]/1000)**2)
    v_n[i] = v

v_n = np.delete(v_n,-1)
sp.Matrix(v_n)

Matrix([
[0.919565213041451],
[ 1.01859641604325],
[ 1.78254864111567],
[ 1.34456008026081]])

In [41]:
v = 4 * Q_n[1]/(math.pi*(d_tuberias[1,0]/1000)**2)
v

1.01859641604325

In [37]:
def Col_White():

array([5.00e+02, 1.74e+02, 1.50e-03, 3.40e+00])